# Machine Learning Lab 8
#### Programmed By Sravanth Chowdary Poturi

Implantation of CNN using PyTorch Tutorial for Image classification.1. Load and normalize any dataset available in PyTorch
2. Design LeNet model using PyTorch.
3. Train and Test the model on the selected dataset
4. Experiment with different number of filters, kernel sizes, number of layers.
5. Compare the results.

Reference Link:[Click Here](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html)


In [1]:
# import libraries
import torch
import torchvision
import torchvision.transforms as transforms

# load and normalize the dataset
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

# define the classes
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

100.0%


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [4]:
# Design LeNet model using PyTorch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


In [ ]:
# Train and Test the model on the selected dataset
import torch.optim as optim

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = Net()

# define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# train the model
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

# test the model on test set
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
# print the accuracy
print('The Accuracy is' % (
    100 * correct / total))

In [ ]:
# Experiment with different number of filters, kernel sizes, number of layers.
# Compare the results.
import matplotlib.pyplot as plt
import numpy as np

# setting the parameters to be used as list
filters = [6, 12, 18, 24]
kernel_sizes = [3, 5, 7, 9]
layers = [2, 3, 4, 5]

# dictionary to store the results
results = {}

# loop over the parameters
for filter in filters:
    for kernel_size in kernel_sizes:
        for layer in layers:
            # define the model
            class Net(nn.Module):
                def __init__(self):
                    super(Net, self).__init__()
                    self.conv1 = nn.Conv2d(3, filter, kernel_size)
                    self.pool = nn.MaxPool2d(2, 2)
                    self.conv2 = nn.Conv2d(filter, 16, kernel_size)
                    self.fc1 = nn.Linear(16 * 5 * 5, 120)
                    self.fc2 = nn.Linear(120, 84)
                    self.fc3 = nn.Linear(84, 10)

                def forward(self, x):
                    x = self.pool(F.relu(self.conv1(x)))
                    x = self.pool(F.relu(self.conv2(x)))
                    x = x.view(-1, 16 * 5 * 5)
                    x = F.relu(self.fc1(x))
                    x = F.relu(self.fc2(x))
                    x = self.fc3(x)
                    return x

                def num_flat_features(self, x):
                    size = x.size()[1:]  # all dimensions except the batch dimension
                    num_features = 1
                    for s in size:
                        num_features *= s
                    return num_features

            # define the loss function and optimizer
            criterion = nn.CrossEntropyLoss()
            optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

            # train the model
            for epoch in range(2):  # loop over the dataset multiple times

                running_loss = 0.0
                for i, data in enumerate(trainloader, 0):
                    # get the inputs
                    inputs, labels = data

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward + backward + optimize
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()

                    # print statistics
                    running_loss += loss.item()
                    if i % 2000 == 1999:  # print every 2000 mini-batches
                        print('[%d, %5d] loss: %.3f' %
                              (epoch + 1, i + 1, running_loss / 2000))
                        running_loss = 0.0

            print('Finished Training')
            # save the results
            results[(filter, kernel_size, layer)] = (100 * correct / total)

# plot the results
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
x = np.array([i[0] for i in results.keys()])
y = np.array([i[1] for i in results.keys()])
z = np.array([i[2] for i in results.keys()])
c = np.array([i for i in results.values()])
ax.scatter(x, y, z, c=c)
ax.set_xlabel('Number of Filters')
ax.set_ylabel('Kernel Size')
ax.set_zlabel('Number of Layers')
plt.show()